### 인공지능 플랫폼 HuggingFace를 활용해 보기
사전 학습된 BERT 모델을 가져온 다음, 넷플릭스 리뷰 데이터를 활용하여 파인 튜닝을 진행하고 그 모델을 활용함.

### 1. 데이터셋 불러오기 및 전처리

In [1]:
import pandas as pd

In [2]:
# 데이터는 dataset/netflix 안에 있다.

df = pd.read_csv('dataset/netflix/netflix_reviews.csv')

In [3]:
import re

# 전처리 함수
def preprocess_text(text):
    if isinstance(text, float):
        return ""
    text = text.lower()  # 대문자를 소문자로
    text = re.sub(r'[^\w\s]', '', text)  # 구두점 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = text.strip()  # 띄어쓰기 제외하고 빈 칸 제거
    return text

In [4]:
# 데이터 불러오고 사소한 전처리를 먼저 함.
df = pd.read_csv('dataset/netflix/netflix_reviews.csv')
df.content = df.content.apply(preprocess_text)

# 각 별점에 따라 10000개씩만 추출
new_df = pd.DataFrame(columns=df.columns)
for i in range(5):
    new_df = pd.concat([new_df, df[df.score == i + 1].sample(10000)])[['content', 'score']]

new_df.columns = ['text', 'label']
new_df

,text,label
65542,useless on my tablet cannot detect any network...,1
8558,some of the movie are good but i have a proble...,1
93000,whats going on the out of sink sound its like ...,1
50568,totally fraud and cheated the said they r givi...,1
103039,latest update wont load in my ppro please fix ...,1
...,...,...
2778,i like the movies documents and series costume...,5
93082,i like netflix because it has so many relaxing...,5
109132,what a cool app but there are some movies that...,5
4463,good enjoy,5


In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from datasets import Dataset
from transformers import AutoTokenizer

# 위의 데이터프레임을 dataset 객체로 바꿔준다.
dataset = Dataset.from_pandas(new_df.reset_index(drop=True))

# HuggingFace의 Model 탭에서 확인할 수 있으며, 해당 모델의 이름 혹은 주소를 알고 있다면
# 손쉽게 모델에 맞는 tokenizer와 사전 훈련된 모델을 바로 불러올 수 있다.
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(dataset):
    # 텍스트 데이터를 토큰화한다.
    # 텍스트 데이터를 모델에 입력할 때, 특히 배치 처리에서는 모든 텍스트의 길이를 통일시켜 주어야 한다.
    # padding은 짧은 문장을 길게 늘일 때 사용하며 목표 길이에 모자란 만큼 0을 추가해 나가는 식으로 동작한다.
    # truncation은 긴 문장을 짧게 줄일 때 사용하며 목표 길이를 초과한 위치에 있는 값들은 사라진다.
    # (단, 맨 끝에 들어가는 special token은 사라지지 않는다. 즉, 입력한 것에 대해서만 토큰을 삭제한다.)
    return tokenizer(dataset['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True).train_test_split(test_size=0.2)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### 2. 모델 훈련

In [6]:
from transformers import AutoModelForSequenceClassification

# score는 1점부터 5점까지이므로 라벨의 개수는 5개.
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer", num_train_epochs=10, eval_strategy="epoch"
)

아래 두 셀은 학습이 어느 정도로 되었다는 생각이 들 때까지 원하는 만큼 반복하면 된다.

In [9]:
# 필요한 경우 일부만 추출해서 할 수도 있다.
small_train_dataset = tokenized_datasets["train"].shuffle(seed=1028).select(range(5000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=1028).select(range(1000))

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.057600,1.008816,0.317000
2,0.920100,0.926772,0.370000
3,0.832900,1.024872,0.344000
4,0.543100,1.288122,0.368000
5,0.368800,1.737536,0.359000
6,0.297200,2.257051,0.375000
7,0.209800,2.756801,0.377000
8,0.080800,3.011005,0.377000
9,0.043200,3.323883,0.367000
10,0.029000,3.373793,0.366000


TrainOutput(global_step=6250, training_loss=0.41950544746398927, metrics={'train_runtime': 3822.8505, 'train_samples_per_second': 13.079, 'train_steps_per_second': 1.635, 'total_flos': 1.31559071232e+16, 'train_loss': 0.41950544746398927, 'epoch': 10.0})

In [22]:
from transformers import pipeline
rating_classification = pipeline('text-classification', model=model, tokenizer=tokenizer, device='mps')

rating_classification("I'm going to subscribe to Disney instead of using this.")

[{'label': 'LABEL_1', 'score': 0.9997639060020447}]

In [17]:
dataset[11111]

{'text': 'dont override my phones brightness controls if your going to have in app brightness controls then use the phones native brightness the lowest setting is still too damn bright',
 'label': 2}

In [31]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [35]:
# 처음에 pretrained 모델을 불러왔듯, 똑같이 pretrained 모델을 저장하면 된다.
# 인자를 이름으로 한 디렉토리가 생성된다.
# model.save_pretrained('BERT_model')

# 불러오기
# t_model = AutoModelForSequenceClassification.from_pretrained('BERT_model')
# t_model